In [45]:
import pandas as pd
import psycopg2
import os
from io import StringIO
from dotenv import load_dotenv

load_dotenv()

df = pd.read_json('task1_d.json')


In [ ]:
def df_to_postgres(df, table_name, connection_params):
    
    conn = psycopg2.connect(**connection_params)
    
    with conn.cursor() as cur:
        cur.execute(f"DROP TABLE IF EXISTS {table_name}") # drop existing table
        
        create_table_sql = f"""
        CREATE TABLE {table_name} (
            {', '.join([f'{col} TEXT' for col in df.columns])} 
        )
        """
        cur.execute(create_table_sql)    # and recreate new
        
        output = StringIO()
        df.to_csv(output, sep='\t', header=False, index=False)
        output.seek(0)
        
        cur.copy_from(output, table_name, null="") # push to postgres
        conn.commit()
    
    conn.close()
    print(f"Successfully wrote {len(df)} rows to {table_name}")

connection_params = {
    'host': os.getenv("PG_HOST"),
    'database': os.getenv("PG_DB_NAME"),
    'user': os.getenv("PG_USER_NAME"),
    'password': os.getenv("PG_PASS")
}

df_to_postgres(df, 'book_sales', connection_params)

Successfully wrote 5003 rows to book_sales
